In [ ]:
import brt
import pandas as pd

In [ ]:
#display all funda columns

brt.show_sas_columns('funda.sas7bdat')

In [ ]:
#read 2,000 observations from funda

keys = ['datafmt','indfmt','consol','popsrc','datadate','gvkey','fyear','sich','sale','ib','at','prcc_f','csho','dltt','dlc','rect','invt','cogs','xrd','capx','ni','oiadp','ppegt','seq','che', 'chech','xsga','emp','act','lct','txp','dp']
funda = brt.read_sas("funda.sas7bdat", keys, chunksize=1000, batch=2)

In [ ]:
#Apply WRDS filter

funda = brt.get_comp(funda)

In [ ]:
#Remove financial firms

funda = brt.remove_sic_between(funda, 'sich',6000,6999)

In [ ]:
#Add Fama-French 5-factor code

brt.add_ff5(funda,'sich')

In [ ]:
#Read crsp

keys=['DATE','PERMNO','RET']
dsf = brt.read_sas("dsf.sas7bdat", keys, chunksize=1000)

In [ ]:
#Merge Compustat and CRSP

ccmxpf_lnkhist = brt.read_sas('ccmxpf_lnkhist.sas7bdat')
funda = brt.add_permno(funda, ccmxpf_lnkhist)
funda['datadate']=pd.to_datetime(funda[ 'datadate']) 
funda_dsf = funda.merge(dsf ,left_on=['PERMNO', 'datadate'], right_on = ['PERMNO', 'DATE'])

In [ ]:
#Winsorize 'sale' and 'ib' columns

funda_dsf = brt.winsorize(funda_dsf.reset_index(drop=True), columns=['sale', 'ib']);